In [21]:
#!/usr/bin/env python3
"""
Multi‑Agent • Single‑Task Demo (Planning • Strategy • Autonomy with Roles)

One task (natural‑language goal) executed by multiple specialized agents:
- PlanningAgent: extract constraints and break the task into steps
- ResearchAgent: (mock) look up heuristics/assumptions
- StrategyAgent: choose channels and budget split
- ContentAgent: generate ideas
- SchedulingAgent: build a day‑by‑day schedule
- ForecastAgent: estimate outcomes
- CriticAgent: review/critique artifacts and suggest fixes
- Orchestrator: routes artifacts on a simple blackboard until done

Run:
    python multi_agent_single_task.py \
      --goal "Plan a 3-day social promo for a sports brand with ₹50,000 budget to boost ecommerce sales." \
      --brand FleetAthlete --theme Speed

Outputs:
- Console summary
- A final brief saved to: final_brief.txt
- A trace log of agent messages: run_trace.json

Note: Heuristics are intentionally simple; swap ResearchAgent to call real APIs/LLMs.
"""
from __future__ import annotations
import argparse
import json
import random
import re
import sys # Import sys module
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional

In [23]:
# -----------------------------
# Shared tools & types
# -----------------------------

class CalculatorTool:
    ALLOWED = set("0123456789.+-*/()% ")
    def run(self, expr: str) -> float:
        if not set(expr) <= self.ALLOWED:
            raise ValueError("Unsafe expression.")
        return float(eval(expr, {"__builtins__": None}, {}))

class KnowledgeBaseTool:
    KB = {
        "cpc_india_sports_paid_social": 6.5,
        "ctr_organic_social": 0.015,
        "conv_rate_ecom": 0.02,
        "avg_order_value": 1800.0,
        "influencer_micro_cost_per_post": 5000.0,
        "email_ctr": 0.03,
        "email_conv": 0.02,
        "best_posting_times": ["7:30 AM", "12:30 PM", "7:30 PM"],
        "channels": ["Paid Social", "Organic Social", "Influencers", "Email/SMS", "Content/Blog"],
    }
    def lookup(self, key: str) -> Any:
        return self.KB.get(key)

@dataclass
class Blackboard:
    goal: str
    brand: str
    theme: str
    data: Dict[str, Any] = field(default_factory=dict)
    notes: List[Dict[str, Any]] = field(default_factory=list)
    def set(self, k: str, v: Any):
        self.data[k] = v
    def get(self, k: str, default=None):
        return self.data.get(k, default)
    def log(self, agent: str, message: str, payload: Optional[Dict[str, Any]] = None):
        self.notes.append({"agent": agent, "message": message, "payload": payload or {}})



In [22]:
# -----------------------------
# Agents
# -----------------------------

class PlanningAgent:
    NAME = "PlanningAgent"
    BUDGET_REGEX = r"(₹|INR|\$)?\s?([0-9][0-9,\.]*)"
    DAYS_REGEX = r"(\d+)\s*(day|days|d)\b"

    def run(self, bb: Blackboard):
        text = bb.goal
        # parse budget (largest number) and days (first match)
        matches = re.findall(self.BUDGET_REGEX, text, flags=re.I)
        nums = []
        for _, raw in matches:
            try:
                nums.append(float(raw.replace(",", "")))
            except:  # noqa: E722
                pass
        budget = max(nums) if nums else 50000.0
        m = re.search(self.DAYS_REGEX, text, flags=re.I)
        days = int(m.group(1)) if m else 3
        steps = [
            "Clarify constraints",
            "Research heuristics",
            "Choose channels & budget split",
            "Generate content ideas",
            "Build schedule",
            "Forecast outcomes",
            "Critique & improve",
            "Assemble final brief",
        ]
        bb.set("constraints", {"budget": budget, "days": days})
        bb.set("steps", steps)
        bb.log(self.NAME, "Parsed constraints and planned steps", {"budget": budget, "days": days, "steps": steps})

class ResearchAgent:
    NAME = "ResearchAgent"
    def __init__(self, kb: KnowledgeBaseTool):
        self.kb = kb
    def run(self, bb: Blackboard):
        heur = {
            "cpc": self.kb.lookup("cpc_india_sports_paid_social"),
            "conv": self.kb.lookup("conv_rate_ecom"),
            "aov": self.kb.lookup("avg_order_value"),
            "influencer_cost": self.kb.lookup("influencer_micro_cost_per_post"),
            "email_ctr": self.kb.lookup("email_ctr"),
            "email_conv": self.kb.lookup("email_conv"),
            "best_times": self.kb.lookup("best_posting_times"),
        }
        bb.set("heuristics", heur)
        bb.log(self.NAME, "Loaded planning heuristics", heur)

class StrategyAgent:
    NAME = "StrategyAgent"
    def run(self, bb: Blackboard):
        budget = bb.get("constraints")["budget"]
        days = bb.get("constraints")["days"]
        if budget < 20000:
            alloc = {"Organic Social": 0.0, "Influencers": 0.6, "Content/Blog": 0.2, "Email/SMS": 0.2}
        elif budget < 100000:
            alloc = {"Paid Social": 0.5, "Influencers": 0.2, "Organic Social": 0.15, "Email/SMS": 0.15}
        else:
            alloc = {"Paid Social": 0.6, "Influencers": 0.2, "Content/Blog": 0.1, "Email/SMS": 0.1}
        split = {k: round(v, 2) for k, v in alloc.items()}
        rupees = {k: round(budget * v) for k, v in split.items()}
        strat = {"allocation": split, "rupees": rupees, "days": days}
        bb.set("strategy", strat)
        bb.log(self.NAME, "Selected channels and budget split", strat)

class ContentAgent:
    NAME = "ContentAgent"
    def brainstorm(self, brand: str, theme: str, n: int = 6) -> List[str]:
        random.seed(42 + len(brand) + len(theme))
        verbs = ["Crush", "Chase", "Own", "Level-Up", "Ignite", "Unlock"]
        angles = ["form tips", "micro-workouts", "gear care", "coach wisdom", "community stories", "before/after"]
        ideas = []
        for _ in range(n * 2):
            v = random.choice(verbs)
            a = random.choice(angles)
            ideas.append(f"{v} your game: {a} ({theme} × {brand})")
        seen, out = set(), []
        for i in ideas:
            if i not in seen:
                seen.add(i)
                out.append(i)
            if len(out) == n:
                break
        return out
    def run(self, bb: Blackboard):
        ideas = self.brainstorm(bb.brand, bb.theme)
        bb.set("ideas", ideas)
        bb.log(self.NAME, "Generated content ideas", {"count": len(ideas)})

class SchedulingAgent:
    NAME = "SchedulingAgent"
    def run(self, bb: Blackboard):
        days = bb.get("constraints")["days"]
        times = bb.get("heuristics")["best_times"] or ["Morning", "Afternoon", "Evening"]
        ideas = bb.get("ideas", [])
        schedule: List[Dict[str, Any]] = []
        for d in range(1, max(1, days) + 1):
            slots = {}
            for t in times:
                slots[t] = [
                    f"Post: {random.choice(ideas) if ideas else 'TBD'}",
                    "Story/Reel: behind-the-scenes",
                ]
            schedule.append({"day": d, "slots": slots})
        bb.set("schedule", schedule)
        bb.log(self.NAME, "Built day-by-day schedule", {"days": days, "slots": len(times)})

class ForecastAgent:
    NAME = "ForecastAgent"
    def __init__(self, kb: KnowledgeBaseTool):
        self.kb = kb
    def run(self, bb: Blackboard):
        strat = bb.get("strategy")
        rupees = strat["rupees"]
        conv = self.kb.lookup("conv_rate_ecom")
        aov = self.kb.lookup("avg_order_value")
        cpc = self.kb.lookup("cpc_india_sports_paid_social")
        infl_cost = self.kb.lookup("influencer_micro_cost_per_post")
        email_ctr = self.kb.lookup("email_ctr")
        email_conv = self.kb.lookup("email_conv")

        results: Dict[str, Dict[str, Any]] = {}
        # Paid Social
        if rupees.get("Paid Social", 0) > 0 and cpc:
            clicks = rupees["Paid Social"] / cpc
            orders = clicks * conv
            revenue = orders * aov
            results["Paid Social"] = {"clicks": int(clicks), "orders": int(orders), "revenue": int(revenue)}
        # Influencers
        if rupees.get("Influencers", 0) > 0 and infl_cost:
            posts = max(1, int(rupees["Influencers"] // infl_cost))
            clicks = posts * 150
            orders = clicks * conv
            revenue = orders * aov
            results["Influencers"] = {"posts": posts, "clicks": int(clicks), "orders": int(orders), "revenue": int(revenue)}
        # Organic
        if "Organic Social" in rupees:
            clicks = 400
            orders = clicks * conv
            results["Organic Social"] = {"clicks": clicks, "orders": int(orders)}
        # Email
        if "Email/SMS" in rupees:
            list_size = 10000
            clicks = list_size * email_ctr
            orders = clicks * email_conv
            revenue = orders * aov
            results["Email/SMS"] = {"clicks": int(clicks), "orders": int(orders), "revenue": int(revenue)}
        totals = {"orders": sum(v.get("orders", 0) for v in results.values()),
                  "revenue": sum(v.get("revenue", 0) for v in results.values())}
        results["TOTALS"] = totals
        bb.set("forecast", results)
        bb.log(self.NAME, "Created rough forecast", totals)

class CriticAgent:
    NAME = "CriticAgent"
    def run(self, bb: Blackboard):
        constraints = bb.get("constraints")
        strat = bb.get("strategy")
        schedule = bb.get("schedule")
        issues: List[str] = []
        # Example checks
        if constraints["budget"] < 20000 and strat["allocation"].get("Paid Social", 0) > 0:
            issues.append("Budget too low for Paid Social; shift to Influencers/Email/Organic.")
        if not schedule or not any(day["slots"] for day in schedule):
            issues.append("Schedule is empty.")
        # Feedback application (simple)
        if issues:
            alloc = strat["allocation"].copy()
            if "Budget too low for Paid Social" in issues[0]:
                spend = strat["rupees"].get("Paid Social", 0)
                alloc.pop("Paid Social", None)
                # reassign equally to Influencers and Email/SMS
                add = {"Influencers": 0.5, "Email/SMS": 0.5}
                # renormalize
                remaining = 1.0 - sum(v for k, v in alloc.items())
                for k, v in add.items():
                    alloc[k] = round(alloc.get(k, 0.0) + remaining * v, 2)
                # recompute rupees
                rupees = {k: round(constraints["budget"] * v) for k, v in alloc.items()}
                strat = {"allocation": alloc, "rupees": rupees, "days": constraints["days"]}
                bb.set("strategy", strat)
        bb.log(self.NAME, "Critique complete", {"issues": issues})

class WriterAgent:
    NAME = "WriterAgent"
    def run(self, bb: Blackboard):
        c = bb.get("constraints")
        strat = bb.get("strategy")
        ideas = bb.get("ideas", [])
        schedule = bb.get("schedule", [])
        forecast = bb.get("forecast", {})
        lines: List[str] = []
        lines.append(f"# Launch Brief\n")
        lines.append(f"**Goal:** {bb.goal}")
        lines.append(f"**Brand/Theme:** {bb.brand} / {bb.theme}")
        lines.append(f"**Constraints:** Budget ₹{int(c['budget'])}, Duration {c['days']} days\n")
        lines.append("## Channel Strategy & Budget Split")
        for k, v in strat["allocation"].items():
            lines.append(f"- {k}: {int(v*100)}% (₹{strat['rupees'][k]})")
        lines.append("\n## Content Ideas (shortlist)")
        for i in ideas:
            lines.append(f"- {i}")
        lines.append("\n## Day-by-Day Schedule")
        for day in schedule:
            lines.append(f"- **Day {day['day']}**")
            for slot, tasks in day["slots"].items():
                for task in tasks:
                    lines.append(f"  - {slot}: {task}")
        lines.append("\n## Rough Forecast (very rough, for planning only)")
        for ch, vals in forecast.items():
            if ch == "TOTALS":
                continue
            pretty = ", ".join(f"{k}={v}" for k, v in vals.items())
            lines.append(f"- {ch}: {pretty}")
        totals = forecast.get("TOTALS", {})
        lines.append(f"- **TOTALS:** orders={totals.get('orders', 0)}, revenue=₹{totals.get('revenue', 0)}")
        lines.append("\n## Risks & Mitigations")
        lines.append("- Creative fatigue → rotate formats daily; reuse top winners only.")
        lines.append("- Tracking issues → verify UTM and pixels before Day 1.")
        lines.append("- Inventory runouts → sync with ops; set real scarcity only.")
        lines.append("\n## Success Metrics")
        lines.append("- CTR, CPC (paid), follower growth (organic), email CTR/conv, total orders & revenue.")
        brief = "\n".join(lines)
        bb.set("brief", brief)
        bb.log(self.NAME, "Assembled final brief", {"length": len(brief)})

In [24]:
# -----------------------------
# Orchestrator
# -----------------------------

class Orchestrator:
    def __init__(self):
        self.kb = KnowledgeBaseTool()
        self.agents = [
            PlanningAgent(),
            ResearchAgent(self.kb),
            StrategyAgent(),
            ContentAgent(),
            SchedulingAgent(),
            ForecastAgent(self.kb),
            CriticAgent(),
            ForecastAgent(self.kb),  # re-run forecast after critique if strategy changed
            WriterAgent(),
        ]
    def run(self, goal: str, brand: str, theme: str) -> Blackboard:
        bb = Blackboard(goal=goal, brand=brand, theme=theme)
        for a in self.agents:
            a.run(bb)
        return bb



In [25]:
# -----------------------------
# CLI
# -----------------------------

def parse_args():
    p = argparse.ArgumentParser(description="Multi-Agent Single-Task Demo")
    p.add_argument("--goal", required=False, help="Natural-language goal for the task")
    p.add_argument("--brand", default="YourSportsBrand")
    p.add_argument("--theme", default="Performance")
    # Allow unknown arguments to be ignored
    args, unknown = p.parse_known_args()
    return args

def main():
    try:
        args = parse_args()
    except SystemExit as e:
        # Handle the SystemExit exception gracefully in Colab
        if e.code != 2: # Exit code 2 usually means invalid arguments. Other codes might be intended exits.
           raise # Re-raise if not an invalid argument error

        # If the goal is not provided, use a default.
        goal = "Plan a 3-day social media promotion for a sports brand with ₹50,000 budget to boost ecommerce sales."
        brand = "YourSportsBrand"
        theme = "Performance"

    else:
      goal = args.goal or "Plan a 3-day social media promotion for a sports brand with ₹50,000 budget to boost ecommerce sales."
      brand = args.brand
      theme = args.theme


    orch = Orchestrator()
    bb = orch.run(goal=goal, brand=brand, theme=theme)

    # Write artifacts
    with open("final_brief.txt", "w", encoding="utf-8") as f:
        f.write(bb.get("brief", ""))
    with open("run_trace.json", "w", encoding="utf-8") as f:
        json.dump(bb.notes, f, indent=2)

    # Console summary
    print("=== MULTI-AGENT • SINGLE TASK SUMMARY ===")
    print("Goal:", bb.goal)
    print("Constraints:", bb.get("constraints"))
    totals = (bb.get("forecast") or {}).get("TOTALS", {})
    print("Forecast Totals:", totals)
    print("Brief written to: final_brief.txt")
    print("Trace log written to: run_trace.json")



In [26]:
if __name__ == "__main__":
    main()

=== MULTI-AGENT • SINGLE TASK SUMMARY ===
Goal: Plan a 3-day social media promotion for a sports brand with ₹50,000 budget to boost ecommerce sales.
Constraints: {'budget': 50000.0, 'days': 3}
Forecast Totals: {'orders': 96, 'revenue': 160061}
Brief written to: final_brief.txt
Trace log written to: run_trace.json
